In [ ]:
import pandas as pd
import tweepy
import json

In [4]:
consumer_key= 'xf96zGbBtmbXnIJBIKbhg7IQ1'
consumer_secret= 'HyHYCA8gB1oGN9H5bMEotatc3AfSAMSCK2SeoFVuTdYVwxTpgY'
access_token= '106489559-rxeRE7aH8f9Z7V50CsHfaYeZ7DrzmaJBAmn8miNi'
access_token_secret= 'lJbQlDcLjOtbZNSyJ1qAVJ7jvBpBEJOHnAOoI1KPaetJj'

In [3]:
import json

# Enter your keys/secrets as strings in the following fields
credentials = {}  
credentials['CONSUMER_KEY'] = 'xf96zGbBtmbXnIJBIKbhg7IQ1'
credentials['CONSUMER_SECRET'] = 'HyHYCA8gB1oGN9H5bMEotatc3AfSAMSCK2SeoFVuTdYVwxTpgY'
credentials['ACCESS_TOKEN'] = '106489559-rxeRE7aH8f9Z7V50CsHfaYeZ7DrzmaJBAmn8miNi'
credentials['ACCESS_SECRET'] = 'lJbQlDcLjOtbZNSyJ1qAVJ7jvBpBEJOHnAOoI1KPaetJj'

# Save the credentials object to file
with open("twitter_credentials.json", "w") as file:  
    json.dump(credentials, file)

In [4]:
# Import the Twython class
from twython import Twython  
import json

# Load credentials from json file
with open("twitter_credentials.json", "r") as file:  
    creds = json.load(file)

# Instantiate an object
python_tweets = Twython(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])

# Create our query
query = {'q': '#refugees', 
        'result_type': 'popular',
        'count': 20,
        'lang': 'en'
        }

In [5]:
import pandas as pd

# Search tweets
dict_ = {'user': [], 'date': [], 'text': [], 'hashtag': [], 'location':[]}  
for status in python_tweets.search(**query)['statuses']:  
    dict_['user'].append(status['user']['screen_name'])
    dict_['date'].append(status['created_at'])
    dict_['text'].append(status['text'])
    dict_['hashtag'].append(status['entities']['hashtags'])
    dict_['location'].append(status['user']['location'])

# Structure data in a pandas DataFrame for easier manipulation
df = pd.DataFrame(dict_)  
#df.sort_values(by='favorite_count', inplace=True, ascending=False)  

#df.head() 



In [30]:
df.shape

(13, 5)

# Using Streamining to obtain more twits using the module "TwythonStreamer" from the library "Twython"

In [6]:
from twython import TwythonStreamer  
import csv

# Filter out unwanted data
def process_tweet(tweet):  
    d = {}
    d['hashtags'] = [hashtag['text'] for hashtag in tweet['entities']['hashtags']]
    d['text'] = tweet['text']
    d['user'] = tweet['user']['screen_name']
    d['user_loc'] = tweet['user']['location']
    return d


# Create a class that inherits TwythonStreamer
class MyStreamer(TwythonStreamer):     

    # Received data
    def on_success(self, data):

        # Only collect tweets in English
        if data['lang'] == 'en':
            tweet_data = process_tweet(data)
            self.save_to_csv(tweet_data)

    # Problem with the API
    def on_error(self, status_code, data):
        print(status_code, data)
        self.disconnect()

    # Save each tweet to csv file
    def save_to_csv(self, tweet):
        with open(r'saved_tweets.csv', 'a') as file:
            writer = csv.writer(file)
            writer.writerow(list(tweet.values()))

In [7]:
# Instantiate from our streaming class
stream = MyStreamer(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'],  
                    creds['ACCESS_TOKEN'], creds['ACCESS_SECRET'])
# Start the stream
stream.statuses.filter(track='refugees')

KeyboardInterrupt: 

In [9]:
import pandas as pd  
headers = ['hashtags', 'tweet', 'user_name', 'location']
tweets = pd.read_csv("saved_tweets.csv", header = None, names = headers)  
tweets.head(50) 


,hashtags,tweet,user_name,location
0,[],RT @QasimRashid: I’m an American Muslim human ...,pghcovert,the steel city
1,[],"This is a morbid article, but thanks.\n\nInvas...",canucklaw1,"British Columbia, Canada"
2,[],RT @BillKristol: I’ve been wondering why the v...,Miss_de_Haro,NaN
3,[],RT @DVATW: Oh look! Calais gets even more enri...,The_Nitr0,NaN
4,[],RT @QasimRashid: I’m an American Muslim human ...,LunaticEuphoric,"Lahore, Pakistan"
5,[],RT @QasimRashid: I’m an American Muslim human ...,ymreiser1,NaN
6,[],RT @Gracie5111: The EU dumped millions of Musl...,CuriousLor,"Huntsville, Ontario"
7,[],RT @BillKristol: I’ve been wondering why the v...,MelindaPowell,Texas
8,[],RT @_SJPeace_: These are PEOPLE who have done ...,uunka2,"London, England"
9,[],RT @MortonAKlein7: Out of concern for the safe...,elisulam,NaN


In [14]:
tweets.shape

(32475, 4)

# Data wrangling

In [16]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32475 entries, 0 to 32474
Data columns (total 4 columns):
hashtags     32475 non-null object
tweet        32475 non-null object
user_name    32475 non-null object
location     20205 non-null object
dtypes: object(4)
memory usage: 1014.9+ KB


In [17]:
list_location_strings = [entry for entry in tweets.location] 

In [20]:
counter_locations = Counter(list_location_strings)

In [92]:
counter_locations.most_common(50)

[(nan, 12270),
 ('United States', 802),
 ('USA', 285),
 ('Turkey', 254),
 ('California, USA', 252),
 ('Texas, USA', 238),
 ('Karabük, Türkiye', 236),
 ('Florida, USA', 195),
 ('Mersin, Turkey', 182),
 ('Canada', 166),
 ('Düzce, Turkey', 155),
 ('Los Angeles, CA', 143),
 ('London, England', 128),
 ('🇺🇸', 120),
 ('North Carolina, USA', 110),
 ('New York, USA', 108),
 ('Washington, DC', 99),
 ('Samsun, Türkiye', 99),
 ('New York, NY', 93),
 ('London', 91),
 ('England, United Kingdom', 90),
 ('Australia', 89),
 ('Chicago, IL', 85),
 ('Darien, IL', 85),
 ('Arizona, USA', 82),
 ('United Kingdom', 81),
 ('Keçiören, Türkiye', 81),
 ('Toronto, Ontario', 75),
 ('Pennsylvania, USA', 73),
 ('Texas', 72),
 ('Michigan, USA', 72),
 ('Adapazarı, Sakarya', 72),
 ('Ohio, USA', 70),
 ('Houston, TX', 69),
 ('Earth', 68),
 ('New Jersey, USA', 63),
 ('Dallas, TX', 62),
 ('Georgia, USA', 61),
 ('San Antonio, TX', 60),
 ('New York', 59),
 ('Missouri, USA', 58),
 ('Florida', 54),
 ('Nairobi, Kenya', 53),
 ('Au

In [60]:
df_location = pd.DataFrame.from_dict(counter_locations, orient='index').reset_index()
df_location = df_location.rename(columns={'index':'locations', 0:'count'})
df_location.head()

,locations,count
0,the steel city,1
1,"British Columbia, Canada",10
2,NaN,12270
3,"Lahore, Pakistan",9
4,"Huntsville, Ontario",1


In [72]:
df_location = df_location.sort_values(['count'], ascending=False)

In [73]:
df_loc_filter = df_location[df_location['count'] > 30]

In [91]:
df_loc_filter = df_loc_filter.dropna(axis=0)

df_loc_filter.head()

,locations,count
17,United States,1207
257,Turkey,254
91,"California, USA",252
98,"Texas, USA",238
905,"Karabük, Türkiye",236


In [82]:
# Eliminating the repeated Unite States entries and sum them to the "Unites States" row index 17. 
# The other two rows where "USA" and one with the USA flag. Index 24 and 951 correspondanly.

df_loc_filter.loc[17,'count'] = 802 + 120 + 285

In [87]:
df_loc_filter = df_loc_filter.drop([24, 951])
df_loc_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73 entries, 17 to 1119
Data columns (total 2 columns):
locations    73 non-null object
count        73 non-null int64
dtypes: int64(1), object(1)
memory usage: 1.7+ KB


In [90]:
df_loc_filter.head()

,locations,count
17,United States,1207
257,Turkey,254
91,"California, USA",252
98,"Texas, USA",238
905,"Karabük, Türkiye",236


In [31]:
from collections import Counter  
import ast

#tweets = pd.read_csv("saved_tweets.csv")

# Extract hashtags and put them in a list
list_hashtag_strings = [entry for entry in tweets.hashtags]  
list_hashtag_lists = ast.literal_eval(','.join(list_hashtag_strings))  
hashtag_list = [ht.lower() for list_ in list_hashtag_lists for ht in list_]

# Count most common hashtags
counter_hashtags = Counter(hashtag_list)  
counter_hashtags.most_common(50) 

[('refugees', 281),
 ('concentrationcamps', 128),
 ('libya', 104),
 ('standwithiraqirefugees', 89),
 ('iceraids', 88),
 ('fakechristian', 78),
 ('migrants', 74),
 ('trump', 62),
 ('internationalsocialforum2019', 60),
 ('worldrefugeeday', 53),
 ('usa', 52),
 ('rohingya', 52),
 ('delaram', 42),
 ('forrefugees', 41),
 ('closethecamps', 35),
 ('racistinchief', 32),
 ('turkey', 30),
 ('iranianrefugeesinturkey', 30),
 ('eu', 30),
 ('germany', 27),
 ('peffer', 27),
 ('midwest', 27),
 ('christians', 26),
 ('evangelicals', 26),
 ('jesus', 26),
 ('china', 25),
 ('psychotherapie', 25),
 ('pkv', 25),
 ('standwithlraqirefugees', 24),
 ('cagedkids', 24),
 ('wherearethechildren', 24),
 ('fucktrump', 24),
 ('humanrights', 22),
 ('abolishice', 22),
 ('individual1', 22),
 ('muellerreport', 22),
 ('immigrants', 21),
 ('politics', 21),
 ('resettlement4iranian', 21),
 ('ice', 21),
 ('france', 19),
 ('sar', 19),
 ('syria', 19),
 ('merkel', 19),
 ('canada', 19),
 ('repeatinghistory', 19),
 ('thingsjesusnever

In [40]:
type(counter_hashtags)

collections.Counter

In [47]:
df = pd.DataFrame.from_dict(counter_hashtags, orient='index').reset_index()
df = df.rename(columns={'index':'hashtags', 0:'count'})
df.head()

,hashtags,count
0,crimesagainsthumanity,3
1,crimesagainstchildren,1
2,humanrightsviolations,2
3,china,25
4,tajoura,9


In [51]:
result = df.sort_values(['count'], ascending=False)

In [56]:
result = result[result['count'] > 30]

In [58]:
result.head()

,hashtags,count
17,refugees,281
64,concentrationcamps,128
59,libya,104
36,standwithiraqirefugees,89
5,iceraids,88


# User with most twits and followers